In [29]:
import pandas as pd
from gcs_utils import get_file_info, upload_to_gcs, delete_file_from_gcs, rename_file_in_gcs, download_file_from_gcs, GCS_BUCKET_NAME
import os
import dotenv

dotenv.load_dotenv()


True

In [30]:
# Get file information
files = get_file_info(GCS_BUCKET_NAME)

# Convert file information to DataFrame
df = pd.DataFrame(files)

In [31]:
print(df)

                                                name      size pages
0                                 99745 - ACCENTURE/         0   N/A
1  99745 - ACCENTURE/1. LATAM One Pack - Contrato...  11097221   302
2             Oferta Comercial Citrix LATAM 2023.pdf   2266218     9


In [32]:
# Filtrar los nombres que no terminan con "/"
df_filtered = df[~df['name'].str.endswith('/')]

# Obtener solo los nombres como una lista
nombres_archivos = df_filtered['name'].tolist()

print("Lista de nombres de archivos:")
print(nombres_archivos)


Lista de nombres de archivos:
['99745 - ACCENTURE/1. LATAM One Pack - Contrato Firmado Completo.pdf', 'Oferta Comercial Citrix LATAM 2023.pdf']


In [ ]:
from langchain_google_community import GCSFileLoader

from langchain_community.document_loaders import PyPDFLoader


def load_pdf(file_path):
    return PyPDFLoader(file_path)


loader = GCSFileLoader(
    project_name=os.getenv("GCP_PROJECT_NAME"), bucket=os.getenv("GCS_BUCKET_NAME") , blob="99745 - ACCENTURE/1. LATAM One Pack - Contrato Firmado Completo.pdf", loader_func=load_pdf)

print(loader)


In [ ]:
loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Cargar el documento
documento = loader.load()

# Crear el divisor de texto
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

# Dividir el documento en chunks
chunks = text_splitter.split_documents(documento)

print(f"El documento se ha dividido en {len(chunks)} chunks.")

# Imprimir los primeros 5 chunks como ejemplo
print("\nPrimeros 5 chunks:")
for i, chunk in enumerate(chunks[:5], 1):
    print(f"\nChunk {i}:")
    print(chunk.page_content[:150] + "...")  # Mostrar los primeros 150 caracteres de cada chunk


In [ ]:
# Imprimir los atributos de un chunk de ejemplo
print("\nAtributos de un chunk:")
if chunks:
    ejemplo_chunk = chunks[0]
    print("1. page_content")
    print("2. metadata")
    print("Otros atributos:")
    for attr in dir(ejemplo_chunk):
        if not attr.startswith('_') and attr not in ['page_content', 'metadata']:
            print(f"3. {attr}")
else:
    print("No hay chunks disponibles para analizar.")